In [1]:
%reload_ext autoreload
%autoreload 2

In [7]:
import sys
sys.path.append("..")

import torch
import models.move_evaluation as me_models
import models.lightning as pl_models
from datasets import Dataset
from torch.utils.data import DataLoader

In [8]:
model = me_models.SentimateNet()
pl_model = pl_models.PLClassifierModel(model)

In [24]:
dataset = Dataset.load_from_disk("../../data/huggingface/my_datasets/sentimate_v1_with_tensors/")
dataset.set_format(type='torch', columns=["tensor", "sentiment"], dtype=torch.float32)
dataset = dataset.rename_column("sentiment", "label")
dataset = dataset.rename_column("tensor", "input")
dataset

Dataset({
    features: ['fen', 'move', 'label', 'input'],
    num_rows: 11089186
})

In [25]:
split = dataset.train_test_split(test_size=0.005)

In [31]:
train_dataloader = DataLoader(split["train"], batch_size=128, shuffle=True)
val_dataloader = DataLoader(split["test"], batch_size=128, shuffle=False)

In [32]:
batch = next(iter(train_dataloader))

In [ ]:
# Trainer